In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import pickle

In [3]:
# data = pd.read_csv("train.data.cvs")
# data.replace([np.inf, -np.inf], np.nan, inplace=True)
# data.fillna(0, inplace=True)

In [2]:
test = pd.read_csv("testset.cvs")
test.replace([np.inf, -np.inf], np.nan, inplace=True)
test.fillna(0, inplace=True)

In [3]:
#cut data set
#data.head(50000).to_csv('traincutted.csv', index=False)

In [4]:
# #STANDARTIZE
# mn = {}
# std = {}
# i = 0
# for col in data.columns.drop(['QID', 'Y']):
#     i += 1
#     if (i % 10 == 0):
#         print(i)
#     c = data[col].append(test[col], ignore_index=True)
#     mn[col] = c.mean()
#     std[col] = c.std()
#     data[col] = pd.Series(map(lambda x: (x - mn[col]) / std[col], data[col]))
#     test[col] = pd.Series(map(lambda x: (x - mn[col]) / std[col], test[col]))

# mny = data['Y'].mean()
# stdy = data['Y'].std()
# data['Y'] = pd.Series(map(lambda x: (x - mny) / stdy, data['Y']))
# pickle.dump(data, open('data.pickle', 'wb'))

In [3]:
data = pickle.load(open('data.pickle', 'rb'))

In [4]:
data.head()

,Y,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X128,X129,X130,X131,X132,X133,X134,X135,X136,QID
0,-0.520076,0.8734,4.874875,-1.140098,-0.697683,0.857758,0.578358,3.352266,-1.276091,-0.727231,...,1.633236,-0.261468,-0.876340,1.313413,-0.147491,-0.516928,-0.007767,-0.023933,-0.005446,1
1,-0.520076,0.8734,-0.359608,1.800658,-0.697683,0.857758,0.578358,-0.369378,1.087985,-0.727231,...,1.633236,-0.261468,-0.875984,1.313413,-0.542176,-0.541259,-0.007767,-0.023933,-0.005446,1
2,-0.802483,0.8734,-0.359608,0.820406,-0.697683,0.857758,0.578358,-0.369378,0.299961,-0.727231,...,-0.014477,-0.372609,-0.875984,-1.538956,-0.114601,1.040235,-0.007767,-0.023933,-0.005446,1
3,-0.520076,0.8734,-0.359608,1.800658,-0.697683,0.857758,0.578358,-0.369378,1.087985,-0.727231,...,1.633236,0.961087,-0.876028,1.308666,-0.542176,-0.516928,-0.007767,-0.023933,-0.005446,1
4,-0.661280,0.8734,-0.359608,1.800658,-0.697683,0.857758,0.578358,-0.369378,1.087985,-0.727231,...,-0.014477,0.294239,-0.870102,0.639589,-0.542176,-0.273622,-0.007767,-0.023933,-0.005446,1


In [6]:
from sklearn.model_selection import train_test_split

Y = data['Y'].values
X = data.drop(['Y', 'QID'], axis=1).values
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.02, random_state=1)


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.02)

In [8]:
from xgboost import XGBRegressor
clf = XGBRegressor(n_estimators=200, max_depth=5, learning_rate=0.3, silent=False)
clf

/home/kirill/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.3, max_delta_step=0, max_depth=5,
       min_child_weight=1, missing=None, n_estimators=200, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=False, subsample=1)

In [9]:
#params = {'n_estimators': 100, 'max_depth' : 3, 'n_jobs': 8, 'verbose': 1}
clf.fit(X, Y)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.3, max_delta_step=0, max_depth=5,
       min_child_weight=1, missing=None, n_estimators=200, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=False, subsample=1)

In [13]:
test_X = test.drop(['Y', 'QID'], axis=1).values
test_X.shape

(241521, 136)

In [14]:
numOfDocs = 5
numOfQueries = len(test.QID.unique())
print(numOfQueries)

with open('___subm.csv', 'w') as output:
    output.write('DocumentId,QueryId\n')
    QIDArray = [0] * (test.QID.max() + 1)
    for (i, row) in test.sort_values('Y', ascending=False).iterrows():
        QID = int(row.QID)
        if (QIDArray[QID] < numOfDocs):
            QIDArray[QID] += 1
            output.write("%d,%d\n" % (i + 1,QID))

2000
